# Convolutional Neural Network

### Importing the libraries

In [6]:
!pip install --upgrade tensorflow keras Pillow

  Attempting uninstall: Pillow
    Found existing installation: Pillow 8.4.0
    Uninstalling Pillow-8.4.0:
      Successfully uninstalled Pillow-8.4.0


ERROR: Could not install packages due to an OSError: [WinError 5] Acesso negado: 'C:\\Users\\drpab\\anaconda3\\Lib\\site-packages\\~il\\_imaging.cp39-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [7]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [8]:
tf.__version__

'2.8.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [9]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [10]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [11]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [12]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

### Step 2 - Pooling

In [13]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [14]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [15]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [16]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [17]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [18]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [22]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 30)

Epoch 1/30
250/250 [==============================] - 308s 1s/step - loss: 0.1842 - accuracy: 0.9251 - val_loss: 0.6765 - val_accuracy: 0.7955
Epoch 2/30
250/250 [==============================] - 49s 196ms/step - loss: 0.1662 - accuracy: 0.9341 - val_loss: 0.6565 - val_accuracy: 0.8040
Epoch 3/30
250/250 [==============================] - 50s 200ms/step - loss: 0.1659 - accuracy: 0.9345 - val_loss: 0.7458 - val_accuracy: 0.7840
Epoch 4/30
250/250 [==============================] - 47s 190ms/step - loss: 0.1528 - accuracy: 0.9409 - val_loss: 0.6935 - val_accuracy: 0.7965
Epoch 5/30
250/250 [==============================] - 46s 184ms/step - loss: 0.1358 - accuracy: 0.9464 - val_loss: 0.6750 - val_accuracy: 0.7970
Epoch 6/30
250/250 [==============================] - 46s 184ms/step - loss: 0.1336 - accuracy: 0.9454 - val_loss: 0.7500 - val_accuracy: 0.8010
Epoch 7/30
250/250 [==============================] - 46s 184ms/step - loss: 0.1351 - accuracy: 0.9476 - val_loss: 0.7873 - val_accu

## Part 4 - Making a single prediction

In [20]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

In [21]:
print(prediction)

dog


## Fim

## Considerações
Acuracy de 80% foi alcançada na epoch 20 onde foi relatava como 0.8095 de precisão 